In [72]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras.applications.xception import Xception, decode_predictions
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

In [83]:
small_input_model = Xception(include_top=False, weights='imagenet', input_shape=(128, 128, 3), pooling='avg')
small_input_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [84]:
base_model = Xception()
base_model.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_9[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [87]:
output = base_model.layers[-1](small_input_model.output)
custom_model = Model(small_input_model.inputs, output)
custom_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 63, 63, 32)   864         input_8[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 63, 63, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 63, 63, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [91]:
def predict(file):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = (img - 127.5) / 127.5
    img = np.expand_dims(img, 0)
    print(img.shape)
    
    y_hat = custom_model.predict(img)
    print(decode_predictions(y_hat, top=5))

In [59]:
predict("../../Cat03.png")

[[('n02123159', 'tiger_cat', 0.5227694), ('n02124075', 'Egyptian_cat', 0.13332494), ('n02123045', 'tabby', 0.1206656), ('n02127052', 'lynx', 0.055982742), ('n02971356', 'carton', 0.0037828374)]]


In [60]:
predict("../../mix.jpg")

[[('n03775546', 'mixing_bowl', 0.97112906), ('n04553703', 'washbasin', 0.002466984), ('n07930864', 'cup', 0.0015806287), ('n02939185', 'caldron', 0.0014361043), ('n03786901', 'mortar', 0.000980322)]]


In [92]:
predict("../../rott.jpg")

(1, 128, 128, 3)
[[('n02106550', 'Rottweiler', 1.0), ('n02101006', 'Gordon_setter', 1.2955677e-08), ('n02107908', 'Appenzeller', 1.8990762e-10), ('n02108551', 'Tibetan_mastiff', 6.248136e-11), ('n02096051', 'Airedale', 3.8314603e-12)]]


In [62]:
predict("../../index.jfif")

[[('n04153751', 'screw', 0.97755116), ('n03804744', 'nail', 0.0023800023), ('n03109150', 'corkscrew', 0.00041206743), ('n02790996', 'barbell', 0.00030720184), ('n03065424', 'coil', 0.00029189265)]]


Transfer the XceptionNet to our task: binary classification

In [69]:
output = Dense(2, activation='softmax')(model.layers[-2].output)
my_model = Model(model.inputs, output)
my_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

1) Freeze all layers but the output and train for 3 epochs

In [71]:
for layer in my_model.layers[:-1]:
    layer.trainable = False
    
# Compile


2) Unfreeze all layers and train for 15 epochs

In [ ]:
for layer in my_model.layers:
    layer.trainable = True
    
# Compile
